In [37]:
import pandas as pd
import cobra
import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import linprog
import scipy
import mip
from copy import deepcopy

import warnings
warnings.filterwarnings('ignore')

import json
def save_dict(data, name):
    with open(name, 'w' ) as file:
        json.dump( data, file )

# Community modeling

In this notebook we will implement a method to create community models of two or more species specific metabolic models using cobrapy.

In [38]:
model_DP = cobra.io.read_sbml_model("models/consistent_DP_SNM.xml")
model_SA = cobra.io.read_sbml_model("models/consistent_iYS854_SNM.xml")
print("Growth: ", model_DP.slim_optimize())
print("Growth: ", model_SA.slim_optimize())

Growth:  0.2823653925322476
Growth:  2.558694612613393


In [39]:
for rec in model_SA.reactions:
    rec.lower_bound = max(rec.lower_bound, -1000)
    rec.upper_bound = min(rec.upper_bound, 1000)

In [40]:
import json
compm_SA = json.loads(open("compm_SA.json").read())
compm_DP = json.loads(open("compm_DP.json").read())
compm = compm_SA.copy()
for key in compm_DP:
    if not key in compm:
        compm[key] = compm_DP[key]


In [41]:
BIOMASS_DP = "Growth" 
BIOMASS_SA = "BIOMASS_iYS_wild_type"
models = [model_DP.copy(), model_SA.copy()]

## Community model growths on SNM3 medium


In [42]:
from community_models import *

In [43]:
model1 = Model(model_DP, BIOMASS_DP)
model2 = Model(model_SA, BIOMASS_SA)
shuttle_community_model = MIP_community_model(model1, model2)

# COMPM - 1:1 

In [51]:
shuttle_community_model.reset_model()
shuttle_community_model.set_medium(compm)
shuttle_community_model.weights = [1,1]
shuttle_community_model.optimize()
shuttle_community_model.x1[shuttle_community_model.obj1]
shuttle_community_model.x2[shuttle_community_model.obj2]

In [52]:
print("MBR Weights 1:1: ", shuttle_community_model.optimize())
print("SA growth: ", shuttle_community_model.x2[shuttle_community_model.obj2].x)
print("DP growth: ", shuttle_community_model.x1[shuttle_community_model.obj1].x)
    

MBR Weights 1:1:  2.5562164580248874
SA growth:  2.5562164580248874
DP growth:  0.0


# COOPM 1:1

In [53]:
shuttle_community_model.reset_model()
shuttle_community_model.weights = [1,1]
shuttle_community_model.set_medium(compm)
COOPM = shuttle_community_model.compute_coopm(enforce_survival=False)

In [54]:
shuttle_community_model.reset_model()
shuttle_community_model.set_medium(COOPM)
shuttle_community_model.weights = [1,1]
shuttle_community_model.optimize()
shuttle_community_model.x1[shuttle_community_model.obj1]
shuttle_community_model.x2[shuttle_community_model.obj2]

In [55]:
print("MBR Weights 1:1: ", shuttle_community_model.optimize())
print("SA growth: ", shuttle_community_model.x2[shuttle_community_model.obj2].x)
print("DP growth: ", shuttle_community_model.x1[shuttle_community_model.obj1].x)

MBR Weights 1:1:  0.25562164580248553
SA growth:  0.25562164580248553
DP growth:  0.0


# COOPM 10:1

In [49]:
shuttle_community_model.reset_model()
shuttle_community_model.set_medium(COOPM)
shuttle_community_model.weights = [10,1]
shuttle_community_model.optimize()
shuttle_community_model.x1[shuttle_community_model.obj1]
shuttle_community_model.x2[shuttle_community_model.obj2]

In [50]:
print("MBR Weights 1:1: ", shuttle_community_model.optimize())
print("SA growth: ", shuttle_community_model.x2[shuttle_community_model.obj2].x)
print("DP growth: ", shuttle_community_model.x1[shuttle_community_model.obj1].x)

MBR Weights 1:1:  2.8118381038273763
SA growth:  0.2556216458024894
DP growth:  0.2556216458024887


# COMPM 10:1

In [58]:
shuttle_community_model.reset_model()
shuttle_community_model.set_medium(compm)
shuttle_community_model.weights = [10,1]
shuttle_community_model.set_medium(compm)
shuttle_community_model.optimize()
shuttle_community_model.x1[shuttle_community_model.obj1]
shuttle_community_model.x2[shuttle_community_model.obj2]

In [59]:
print("MBR Weights 1:1: ", shuttle_community_model.optimize())
print("SA growth: ", shuttle_community_model.x2[shuttle_community_model.obj2].x)
print("DP growth: ", shuttle_community_model.x1[shuttle_community_model.obj1].x)
    

MBR Weights 1:1:  2.905058488510848
SA growth:  2.170373335065145
DP growth:  0.07346851534457029


# COOPM 10:1

In [66]:
shuttle_community_model.reset_model()
shuttle_community_model.weights = [10,1]
shuttle_community_model.set_medium(compm)
COOPM = shuttle_community_model.compute_coopm(enforce_survival=True)

In [67]:
shuttle_community_model.reset_model()
shuttle_community_model.set_medium(COOPM)
shuttle_community_model.weights = [10,1]
shuttle_community_model.optimize()
growths_DP.append(shuttle_community_model.x1[shuttle_community_model.obj1])
growths_SA.append(shuttle_community_model.x2[shuttle_community_model.obj2])

In [68]:
print("MBR Weights 1:1: ", shuttle_community_model.optimize())
print("SA growth: ", shuttle_community_model.x2[shuttle_community_model.obj2].x)
print("DP growth: ", shuttle_community_model.x1[shuttle_community_model.obj1].x)

MBR Weights 1:1:  0.581011697702156
SA growth:  0.2905058488509988
DP growth:  0.029050584885115718


# COOPM 1:1

In [69]:
shuttle_community_model.reset_model()
shuttle_community_model.set_medium(COOPM)
shuttle_community_model.weights = [1,1]
shuttle_community_model.optimize()
growths_DP.append(shuttle_community_model.x1[shuttle_community_model.obj1])
growths_SA.append(shuttle_community_model.x2[shuttle_community_model.obj2])

In [70]:
print("MBR Weights 1:1: ", shuttle_community_model.optimize())
print("SA growth: ", shuttle_community_model.x2[shuttle_community_model.obj2].x)
print("DP growth: ", shuttle_community_model.x1[shuttle_community_model.obj1].x)

MBR Weights 1:1:  0.3195564337361145
SA growth:  0.2905058488509988
DP growth:  0.029050584885115718
